In [4]:
# Imports
import sys
import numpy as np

# Print options
np.set_printoptions(threshold=sys.maxsize)

# Sanity check data
grid_world = np.load('data/gridworld.npy')
solution_task31 = np.load('data/stateValuesTask1.npy')

In [18]:
# Global variables
DISCOUNT = 0.9
THRESHOLD = 0.007
GOAL_S = {'x':8,'y':3}
actions = [('r', 0.625, u'\u2192'), ('l', 0.125, u'\u2190'), ('u', 0.125, u'\u2191'), ('d', 0.125, u'\u2193')]
actions_diag = [('r', 0.5625, u'\u2192'), ('l', 0.0625, u'\u2190'), ('u', 0.0625, u'\u2191'), ('d', 0.0625, u'\u2193'), 
                ('lu', 0.0625, u'\u2196'), ('ru', 0.0625, u'\u2197'), ('rd', 0.0625, u'\u2198'), ('ld', 0.0625, u'\u2199')]

# Common routines
def is_outside(state):
    return ((state['y'] < 0 or state['y'] >= grid_world.shape[0]) 
              or (state['x'] < 0 or state['x'] >= grid_world.shape[1]))

def terminal_state(state, goal_inc=True):
    # Entered a goal state
    if goal_inc and state == GOAL_S:
        return True
    # Exeeded boundary limits
    elif is_outside(state):
        return True
    # Entered a cell X
    elif grid_world[state['y'], state['x']] == -20:
        return True

    # Not a terminal state
    else:
        return False

def get_reward(state):
    if is_outside(state):
        return -30;
    else:
        return grid_world[state['y'], state['x']]

def move(state, action, correction=True):
    state_c = state.copy()
    
    if action == 'l':
        state_c['x']  -= 1
    elif action == 'r':
        state_c['x']  += 1
    elif action == 'u':
        state_c['y']  -= 1
    elif action == 'd':
        state_c['y']  += 1
        
    if not terminal_state(state_c, goal_inc=False) or not correction:
        state['x'] = state_c['x']
        state['y'] = state_c['y']
        
    return get_reward(state_c)

def extended_move(state, action, correction=True):
    state_c = state.copy()
    
    if action == 'l':
        state_c['x'] -= 1
    elif action == 'r':
        state_c['x'] += 1
    elif action == 'u':
        state_c['y'] -= 1
    elif action == 'd':
        state_c['y'] += 1
    elif action == 'lu':
        state_c['x'] -= 1
        state_c['y'] -= 1
    elif action == 'ru':
        state_c['x'] += 1
        state_c['y'] -= 1
    elif action == 'ld':
        state_c['x'] -= 1
        state_c['y'] += 1
    elif action == 'rd':
        state_c['x'] += 1
        state_c['y'] += 1
        
    if not terminal_state(state_c, goal_inc=False) or not correction:
        state['x'] = state_c['x']
        state['y'] = state_c['y']
        
    return get_reward(state_c)

def get_actions(action):
    if action == 'l':
        return ['l', 'lu', 'ld']
    if action == 'r':
        return ['r', 'ru', 'rd']
    if action == 'u':
        return ['u', 'ru', 'lu']
    if action == 'd':
        return ['d', 'ld', 'rd']
    if action == 'lu':
        return ['lu', 'l', 'u']
    if action == 'ld':
        return ['ld', 'l', 'd']
    if action == 'ru':
        return ['ru', 'r', 'u']
    if action == 'rd':
        return ['rd', 'r', 'd']

def non_deterministic_move(state, action, V):
    #print("Original state: ", state)
    
    # Transitions probabilities
    transitions = [0.8, 0.1, 0.1]
    
    # Rewards
    rewards = []
    
    # States values
    state_values = []
    
    # Get possible actions
    actions = get_actions(action)
    
    # Populate actions holders
    for l_action in actions:
        # Copy of state
        state_a = state.copy()
        
        # Populate lists
        reward = extended_move(state_a, l_action)
        value = V[state_a['y'], state_a['x']]
        rewards.append(reward)
        state_values.append(value)
        #print("Action ", l_action, " State obtained ", state_a, " Reward ", reward, " Value ", value)
    
    return (transitions, rewards, state_values)
    
def compute_state_value_function(actions, diag_move=False, deterministic=True, iter_lim=np.inf):
    # Value function matrix
    V = np.zeros_like(grid_world)
    
    # Delta value
    delta = np.inf
    
    count=0
    
    while delta > THRESHOLD and count < iter_lim:
        # Reset delta
        delta  = 0

        # Copy of V
        V_copy = V.copy()
        
        count += 1
        
        # States loop
        for y in range(grid_world.shape[0]):
            for x in range(grid_world.shape[1]):
                # Current state
                state = {'x':x,'y':y}
                
                if terminal_state(state):
                    continue
        
                # Temporary value for the state
                V_prev = V[y,x]
                
                # Reset value
                V[y,x] = 0
                
                # Compute new value (diag move allowed)
                for action, probability, _ in actions:
                    state_a = state.copy()
                    
                    if (not deterministic):
                        trans, rewards, state_values = non_deterministic_move(state_a, action, V_copy)
                        for z in range(3):
                            V[y,x] += probability * (trans[z] * (rewards[z] + DISCOUNT * state_values[z]))
                    else:
                        reward = extended_move(state_a, action) if diag_move else move(state_a, action)
                        V[y,x] += probability * (reward + DISCOUNT * V_copy[state_a['y'], state_a['x']])
                
                # Compute new delta
                delta = max(delta, np.abs(V_prev - V[y,x]))

    return V

def policy(state_value_table, actions, diag_move=False):
    # State-Value and Action-Value
    # tables to be used and computed
    V = state_value_table.copy()
    policy = V.copy().astype(object)

    # Set goal cell value to inf
    V[GOAL_S['y'], GOAL_S['x']] = np.inf

    # Loop over every state
    for y in range(grid_world.shape[0]):
        for x in range(grid_world.shape[1]):
            # Current state
            state = {'x':x,'y':y}
            
            # State policy
            state_policy = ''

            # Check if state is Goal
            if state == GOAL_S:
                policy[state['y'],state['x']] = '@'
                continue
                
            # Check if terminal state
            if terminal_state(state, goal_inc=True):
                policy[state['y'],state['x']] = 'X'
                continue

            # Get the best action
            best_r_action = -np.inf
            for action, _, arrow in actions:
                state_a = state.copy()
                reward = extended_move(state_a, action, correction=False) if diag_move else move(state_a, action, correction=False)

                if terminal_state(state_a, goal_inc=False):
                    continue 

                a_move = V[state_a['y'], state_a['x']]
                best_r_action = a_move if a_move > best_r_action else best_r_action

            # Assign arrow(s) (it can be that multiple
            # states are sharing the same reward)
            for action, _, arrow in actions:
                state_a = state.copy()
                reward = extended_move(state_a, action) if diag_move else move(state_a, action)

                if terminal_state(state_a, goal_inc=False):
                    continue 

                if V[state_a['y'], state_a['x']] == best_r_action:
                    state_policy += arrow

            # Insert the arrow in the policy
            policy[state['y'],state['x']] = state_policy

    return policy

## Exercise 3.1

#### Expected value

In [21]:
compute_state_value_function(actions, iter_lim=1)

array([[ -1.375,   3.625,  -3.25 ,  -3.25 ,  -4.625,  -7.   ,   2.25 ,
         -0.125, -20.   ],
       [ -3.25 ,  -2.   ,  -2.   ,  -3.375, -15.25 ,   0.   , -15.25 ,
          0.   , -20.125],
       [-16.5  ,   0.   ,   0.   ,   0.   ,   0.   ,   0.   , -17.625,
          0.   ,  -7.5  ],
       [-16.5  ,   0.   ,   2.5  ,   2.5  ,   2.5  , -16.25 ,   0.   ,
         57.375,   0.   ],
       [ -4.625,   3.5  , -11.5  ,   0.   ,   0.   ,   0.   ,  -5.75 ,
         -3.375,  -6.5  ],
       [ -4.625,  -3.375,  -2.   ,  -5.75 ,  -5.75 ,  -5.75 , -15.25 ,
          0.   , -21.5  ],
       [-16.5  ,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ,   0.   ,
         -5.75 , -21.5  ],
       [  2.25 ,   4.875,   6.25 ,   6.25 ,   2.5  ,   6.25 ,  -0.625,
        -11.5  ,   0.   ],
       [ -1.375,   3.625,   5.   , -13.75 ,   0.   ,   1.25 ,  -1.875,
        -15.125,   0.   ]])

## Task 3.2

#### Optimal value $V^*(s)$

In [15]:
result_task31 = compute_state_value_function(actions)
result_task31

array([[ -55.72233031,  -60.28711964,  -76.2187774 ,  -85.94227143,
         -95.17265171, -102.06584518, -109.1497957 , -128.03694587,
        -154.35820998],
       [ -70.23910801,  -71.03512149,  -82.42332396,  -97.04567539,
        -113.44958832,    0.        , -138.84246494,    0.        ,
        -140.12747124],
       [-114.35530129,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        , -156.42672419,    0.        ,
         -71.57989163],
       [-113.49035193,    0.        ,  -67.70838784,  -82.25522086,
        -104.2432133 , -131.62855066,    0.        ,   67.97547745,
           0.        ],
       [ -66.87419882,  -61.7022072 ,  -77.43326267,    0.        ,
           0.        ,    0.        ,  -51.56820765,  -41.72566933,
         -63.11667048],
       [ -64.48159917,  -63.86372895,  -69.52992994,  -77.89104824,
         -83.19067425,  -88.81991069,  -95.51570805,    0.        ,
        -145.95801304],
       [ -81.40779974,    0.        ,   

The cumulative difference between our result and the "ground truth" result is:

In [16]:
sum(abs(solution_task31 - result_task31).flatten())

0.34249411007374775

#### Optimal policy $\pi^*(s)$

In [7]:
policy(result_task31, actions)

array([['→', '←', '←', '←', '←', '←', '←', '←', '←'],
       ['↑', '↑', '←', '←', '↑', 'X', '↑', 'X', '↓'],
       ['↑', 'X', 'X', 'X', 'X', 'X', '↑', 'X', '↓'],
       ['↓', 'X', '↓', '←', '←', '←', 'X', '→', '@'],
       ['→', '↓', '←', 'X', 'X', 'X', '→', '↑', '↑'],
       ['→', '↑', '←', '←', '←', '←', '↑', 'X', '↑'],
       ['↓', 'X', 'X', 'X', 'X', 'X', 'X', '↓', '↑'],
       ['→', '←', '←', '←', '←', '←', '←', '←', 'X'],
       ['↑', '↑', '↑', '↑', 'X', '↑', '←', '←', 'X']], dtype=object)

## Task 3.3
Actions extended allowing diagonal moves, such that the agent can move to its eight neighboring cells.

#### Optimal value $V^*(s)$
The Policy iteration algoritm changed the value state function table (compared to the task 3.2) by decreasing the value of almost every cell, and this is because the agent has now the possibility to explore the environment with more moves, which ultimately increases the number of time it incours in negative rewards due to how the environment is set.

In [24]:
result_task33=compute_state_value_function(actions_diag, diag_move=True)
result_task33

array([[ -78.85295654,  -84.78663318, -100.82983448, -111.41432921,
        -122.39915622, -132.93539283, -144.82161683, -170.07232622,
        -207.24509049],
       [ -87.24992879,  -88.45634884,  -98.81538951, -113.13610521,
        -129.96200609,    0.        , -140.33734441,    0.        ,
        -179.61116839],
       [-131.36156914,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        , -104.20498146,    0.        ,
         -90.89994106],
       [-126.45709436,    0.        ,  -67.15456507,  -77.81474247,
         -94.73275553, -112.8608242 ,    0.        ,   35.28098586,
           0.        ],
       [ -76.35713138,  -70.98274204,  -85.70573896,    0.        ,
           0.        ,    0.        ,  -60.32823267,  -59.43878516,
         -78.48338783],
       [ -76.1775224 ,  -75.05042911,  -81.76856604,  -89.43287221,
         -96.023486  ,  -99.67864854, -110.44750388,    0.        ,
        -148.21199079],
       [ -91.26110919,    0.        ,   

#### Optimal policy $\pi^*(s)$
The current policy is more flexible and successful than the policy in task 3.2 because now the agent is allowed to navigate routes that were previously forbidden and because the number of steps required to reach the destination is now reduced.

In [11]:
policy(result_task33, actions_diag, diag_move=True)

array([['→', '←', '←', '↙', '←', '←', '←', '↙', '←'],
       ['↑', '↖', '↖', '←', '↖', 'X', '↓', 'X', '↓'],
       ['↑', 'X', 'X', 'X', 'X', 'X', '↘', 'X', '↓'],
       ['↘', 'X', '↙', '←', '←', '↘', 'X', '→', '@'],
       ['→', '↗', '↑', 'X', 'X', 'X', '↗', '↗', '↑'],
       ['↗', '↑', '↖', '←', '←', '↗', '↗', 'X', '↖'],
       ['↘', 'X', 'X', 'X', 'X', 'X', 'X', '↖', '↙'],
       ['→', '←', '←', '←', '←', '←', '←', '←', 'X'],
       ['↗', '↑', '↖', '↖', 'X', '↖', '↖', '↖', 'X']], dtype=object)

## Task 3.4

#### Optimal value $V^*(s)$
The Policy iteration algoritm changed the value state function table (compared to the task 3.3) by increasing the value of the cells around the goal cell and by decreasing the value of those around the border.

In [12]:
result_task34 = compute_state_value_function(actions_diag, diag_move=True, deterministic=False)
result_task34

array([[ -86.03140441,  -89.9602198 , -104.64938837, -114.89711383,
        -125.30481492, -133.83940365, -145.40422954, -168.42400583,
        -206.9191005 ],
       [ -88.74772565,  -89.94085533, -100.48055425, -113.46949619,
        -129.02650581,    0.        , -136.31702458,    0.        ,
        -180.02396947],
       [-120.10731729,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,  -82.11356971,    0.        ,
         -94.84346552],
       [-110.72060201,    0.        ,  -62.83646447,  -68.45619406,
         -78.82369046,  -88.95581937,    0.        ,   16.85273294,
           0.        ],
       [ -72.72754865,  -64.51986164,  -76.87092169,    0.        ,
           0.        ,    0.        ,  -52.5131928 ,  -54.17135432,
         -80.57906673],
       [ -73.51265094,  -72.09177902,  -80.60962022,  -87.08490005,
         -90.68742038,  -88.88222659,  -99.89407572,    0.        ,
        -147.1593049 ],
       [ -79.29326108,    0.        ,   

#### Optimal policy $\pi^*(s)$
The current policy didn't changed significaly compared to the policy of task 3.3, but as it possible to notice in the cell x:6, y:5 (starting to count from 0) the arrow changed from '↗', to '↑' increasing by one the number of required steps to reach the destina

In [13]:
policy(result_task34, actions_diag, diag_move=True)

array([['↓', '←', '↙', '↙', '↙', '←', '←', '↙', '←'],
       ['↑', '↖', '←', '←', '←', 'X', '↓', 'X', '↓'],
       ['↑', 'X', 'X', 'X', 'X', 'X', '↘', 'X', '↓'],
       ['↘', 'X', '↙', '←', '←', '↘', 'X', '→', '@'],
       ['→', '↗', '↑', 'X', 'X', 'X', '↗', '↗', '↑'],
       ['↗', '↑', '↖', '↖', '←', '↗', '↑', 'X', '↖'],
       ['↘', 'X', 'X', 'X', 'X', 'X', 'X', '↖', '↙'],
       ['→', '←', '←', '←', '←', '←', '←', '←', 'X'],
       ['↗', '↑', '↖', '↖', 'X', '↖', '↖', '↖', 'X']], dtype=object)

In [25]:
policy(result_task33, actions_diag, diag_move=True)

array([['→', '←', '←', '↙', '←', '←', '←', '↙', '←'],
       ['↑', '↖', '↖', '←', '↖', 'X', '↓', 'X', '↓'],
       ['↑', 'X', 'X', 'X', 'X', 'X', '↘', 'X', '↓'],
       ['↘', 'X', '↙', '←', '←', '↘', 'X', '→', '@'],
       ['→', '↗', '↑', 'X', 'X', 'X', '↗', '↗', '↑'],
       ['↗', '↑', '↖', '←', '←', '↗', '↗', 'X', '↖'],
       ['↘', 'X', 'X', 'X', 'X', 'X', 'X', '↖', '↙'],
       ['→', '←', '←', '←', '←', '←', '←', '←', 'X'],
       ['↗', '↑', '↖', '↖', 'X', '↖', '↖', '↖', 'X']], dtype=object)